In [117]:
from IPython.display import display
import pandas as pd 
import pickle
import os 
from env import EvoD4jEnv
import re 

env = EvoD4jEnv("Lang", "1", "newTS")
example_num = 3
try_no = 1

# Original
path = "./result/Lang-1b/generated_test/newTS"

# Read Evousite test metadata (columns=['dir', 'evo_relpath', 'evo_test_no', 'evo_test_src', 'evo_target_method'])
with open(os.path.join(path,"evo_test_df.pkl"),'rb') as f:
    evo_test_df = pickle.load(f)

evo_test_df["ground_truth"] = "correct"
display(evo_test_df.head())

,dir,evo_relpath,evo_test_no,evo_test_src,evo_target_method,ground_truth
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test0,@Test(timeout = 4000)\n public void test0() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test1,@Test(timeout = 4000)\n public void test1() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test2,@Test(timeout = 4000)\n public void test2() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test3,@Test(timeout = 4000)\n public void test3() ...,org.apache.commons.lang3.StringUtils.<init>()V,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test0,@Test(timeout = 4000)\n public void test00() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct


In [118]:

prompt_mut_dir = "./result/Lang-1b/generated_test/newTS/prompt_mut/example{}".format(example_num)
prompt_mut_list = list(filter(lambda x: x.endswith('.pkl'), os.listdir(prompt_mut_dir)))
prompt_mut_list.sort()

to_be_appended = pd.DataFrame(columns=['dir', 'evo_relpath', 'evo_test_no', 'evo_test_src', 'evo_target_method', 'ground_truth'])
for prompt_mut_f in prompt_mut_list:
    split_prompt_mut_f = prompt_mut_f.strip().split('_')
    dir = split_prompt_mut_f[0].replace('.','/')
    test_no = split_prompt_mut_f[1]
    tmp_df = evo_test_df.loc[ (evo_test_df["dir"] == dir) & (evo_test_df["evo_test_no"] == test_no),:].copy()
    tmp_df.loc[:,"ground_truth"] = "incorrect"
    to_be_appended = pd.concat([to_be_appended, tmp_df])
evo_test_df = pd.concat([evo_test_df, to_be_appended])
    
display(evo_test_df)


,dir,evo_relpath,evo_test_no,evo_test_src,evo_target_method,ground_truth
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test0,@Test(timeout = 4000)\n public void test0() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test1,@Test(timeout = 4000)\n public void test1() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test2,@Test(timeout = 4000)\n public void test2() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test3,@Test(timeout = 4000)\n public void test3() ...,org.apache.commons.lang3.StringUtils.<init>()V,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test0,@Test(timeout = 4000)\n public void test00() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test1,@Test(timeout = 4000)\n public void test01() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test2,@Test(timeout = 4000)\n public void test02() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test3,@Test(timeout = 4000)\n public void test03() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test4,@Test(timeout = 4000)\n public void test04() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test5,@Test(timeout = 4000)\n public void test05() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct


Parsing Chat_Reply

In [119]:
chat_reply_dir = "./result/Lang-1b/generated_test/newTS/chat_reply/example{}/try{}".format(example_num, try_no)
chat_reply_list = os.listdir(chat_reply_dir)

for chat_reply_f in chat_reply_list:
    with open(os.path.join(chat_reply_dir, chat_reply_f),'r') as fr:
        data = fr.read()
    split_chat_reply_f = chat_reply_f.strip().split('_')
    dir = split_chat_reply_f[0].replace('.','/')
    test_no = split_chat_reply_f[1]
    if re.search("incorrect", data):
        evo_test_df.loc[ (evo_test_df["dir"] == dir) & (evo_test_df["evo_test_no"] == test_no) & (evo_test_df["ground_truth"] == "correct"), "predict" ] = "incorrect"
    elif re.search("correct", data):
        evo_test_df.loc[ (evo_test_df["dir"] == dir) & (evo_test_df["evo_test_no"] == test_no) & (evo_test_df["ground_truth"] == "correct"), "predict" ] = "correct" 

display(evo_test_df)


,dir,evo_relpath,evo_test_no,evo_test_src,evo_target_method,ground_truth,predict
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test0,@Test(timeout = 4000)\n public void test0() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test1,@Test(timeout = 4000)\n public void test1() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test2,@Test(timeout = 4000)\n public void test2() ...,org.apache.commons.lang3.StringUtils.isBlank(L...,correct,correct
0,org/apache/commons/lang3,org/apache/commons/lang3/StringUtils_ESTest.java,test3,@Test(timeout = 4000)\n public void test3() ...,org.apache.commons.lang3.StringUtils.<init>()V,correct,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test0,@Test(timeout = 4000)\n public void test00() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test1,@Test(timeout = 4000)\n public void test01() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test2,@Test(timeout = 4000)\n public void test02() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test3,@Test(timeout = 4000)\n public void test03() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct,incorrect
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test4,@Test(timeout = 4000)\n public void test04() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct,correct
0,org/apache/commons/lang3/math,org/apache/commons/lang3/math/NumberUtils_ESTe...,test5,@Test(timeout = 4000)\n public void test05() ...,org.apache.commons.lang3.math.NumberUtils.crea...,correct,correct


Parsing Chat_Reply_Mut

In [145]:
chat_reply_mut_dir = "./result/Lang-1b/generated_test/newTS/chat_reply_mut/example{}/try{}".format(example_num, try_no)
chat_reply_mut_list = os.listdir(chat_reply_mut_dir)

for chat_reply_mut_f in chat_reply_mut_list:
    with open(os.path.join(chat_reply_mut_dir, chat_reply_mut_f),'r') as fr:
        data = fr.read()
    split_chat_reply_mut_f = chat_reply_mut_f.strip().split('_')
    dir = split_chat_reply_mut_f[0].replace('.','/')
    test_no = split_chat_reply_mut_f[1]
    if re.search("incorrect", data):
        evo_test_df.loc[ (evo_test_df["dir"] == dir) & (evo_test_df["evo_test_no"] == test_no) & (evo_test_df["ground_truth"] == "incorrect"), "predict" ] = "incorrect"
    elif re.search("correct", data):
        evo_test_df.loc[ (evo_test_df["dir"] == dir) & (evo_test_df["evo_test_no"] == test_no) & (evo_test_df["ground_truth"] == "incorrect"), "predict" ] = "correct" 

display(evo_test_df[["evo_test_no","ground_truth","predict"]])


,evo_test_no,ground_truth,predict
0,test0,correct,correct
0,test1,correct,correct
0,test2,correct,correct
0,test3,correct,correct
0,test0,correct,correct
0,test1,correct,correct
0,test2,correct,correct
0,test3,correct,incorrect
0,test4,correct,correct
0,test5,correct,correct


In [134]:
ground_truth = evo_test_df["ground_truth"].values.tolist()
ground_truth = [ 1 if x == "correct" else 0 for x in ground_truth]

predict = evo_test_df["predict"].values.tolist()
predict = [ 1 if x == "correct" else 0 for x in predict]
print(ground_truth)
print(predict)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]


In [140]:
from sklearn import metrics

tn, fp, fn, tp = metrics.confusion_matrix(ground_truth, predict).ravel()

print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)
print("True Positive:", tp)

accuracy = metrics.accuracy_score(ground_truth, predict)
print("accuracy:", accuracy)

precision = metrics.precision_score(ground_truth, predict)
print("precision:", precision)

recall = metrics.recall_score(ground_truth, predict)
print("recall:", recall)

f1 = metrics.f1_score(ground_truth, predict)
print("f1:", f1)

True Negative: 5
False Positive: 5
False Negative: 6
True Positive: 26
accuracy: 0.7380952380952381
precision: 0.8387096774193549
recall: 0.8125
f1: 0.8253968253968254
